In [15]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7982
WEBHOOK_BASE = ""
ADMIN_URL = "http://the_wallet-agent:7981"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [16]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


In [18]:
credential_id = "Operation 0"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)

{'referent': 'Operation 0', 'attrs': {'volt': '10', 'dose': '10', 'speed': '10', 'type': 'drug_A'}, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.1', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:42:default', 'rev_reg_id': None, 'cred_rev_id': None}


### You can get  credential by your credential id ref(seller notebook)

In [10]:
response = await agent_controller.terminate()
print(response)

None
